In [2]:
import datetime
import io
import sys
import unicodedata
from datetime import timezone

import numpy as np
import polars as pl
import pandas as pd
import pybaseball
import requests
import scipy.stats as stats
from bs4 import BeautifulSoup
from pybaseball import statcast
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from keras.models import Model
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from keras.regularizers import l2

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import pybaseball
import requests
import scipy.stats as stats
from bs4 import BeautifulSoup
from pybaseball import statcast
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error, r2_score


pybaseball.cache.enable()

# Load in my own 40 man Roster Scraper
directory = '/Users/cbkaplinger/Desktop/Desktop - Cameron MacBook Pro/Random-Projects/MLB/RosterScraper'
sys.path.append(directory)
from RosterScraper import RosterScraper

## Loads in StatCast ID so batter names show in the Statcast data and loads in a scraped DF with every 40 man roster

In [3]:
url = 'https://docs.google.com/spreadsheets/d/1JgczhD5VDQ1EiXqVG-blttZcVwbZd5_Ne_mefUGwJnk/pub?output=csv'
res = requests.get(url)
ID = pl.read_csv(io.BytesIO(res.content))
ID = ID.drop_nulls(subset=['MLBID'])
ID = ID.with_columns([pl.col('MLBID').cast(pl.Int64)])

Rosters = RosterScraper()
BID = Rosters[Rosters["Position"] == "Batter"]
PID = Rosters[Rosters["Position"] == "Pitcher"]

## Creating functions for data manipulation so they can match when joining separate datasets

In [49]:
def convert_name(name):
    if name == 'Rockies':
        return 'COL'
    elif name == 'Reds':
        return 'CIN'
    elif name == 'Mariners':
        return 'SEA'
    elif name == 'Nationals':
        return 'WSH'
    elif name == 'Yankees':
        return 'NYY'
    elif name == 'Astros':
        return 'HOU'
    elif name == 'Red Sox':
        return 'BOS'
    elif name == 'Athletics':
        return 'OAK'
    elif name == 'Mets':
        return 'NYM'
    elif name == 'Braves':
        return 'ATL'
    elif name == 'Giants':
        return 'SF'
    elif name == 'Brewers':
        return 'MIL'
    elif name == 'Rays':
        return 'TB'
    elif name == 'Royals':
        return 'KC'
    elif name == 'White Sox':
        return 'CWS'
    elif name == 'Cubs':
        return 'CHC'
    elif name == 'Angels':
        return 'LAA'
    elif name == 'Tigers':
        return 'DET'
    elif name == 'Diamondbacks':
        return 'ARI'
    elif name == 'Guardians':
        return 'CLE'
    elif name == 'Orioles':
        return 'BAL'
    elif name == 'Twins':
        return 'MIN'
    elif name == 'Marlins':
        return 'MIA'
    elif name == 'Phillies':
        return 'PHI'
    elif name == 'Rangers':
        return 'TEX'
    elif name == 'Dodgers':
        return 'LAD'
    elif name == 'Padres':
        return 'SD'
    elif name == 'Pirates':
        return 'PIT'
    elif name == 'Blue Jays':
        return 'TOR'
    elif name == 'Cardinals':
        return 'STL'
    else:
        return np.nan
    
def flip_names(name):
    first_name, last_name = name.split(", ")
    return f"{last_name} {first_name}"

def replace_special_chars(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')

def append_suffix_to_duplicates(df, column):
        seen = {}
        for idx, value in enumerate(df[column]):
            if value in seen:
                seen[value] += 1
                df.at[idx, column] = f"{value}2"
            else:
                seen[value] = 1

## Scraping the RotoGrinders website for daily pitchers and lineups

In [50]:
def getDKData2024():
    eastern_time = datetime.datetime.now(timezone.utc).astimezone(timezone(datetime.timedelta(hours=-5)))
    todaysdate = eastern_time.strftime("%m-%d-%Y")
    url = 'https://rotogrinders.com/lineups/mlb?site=draftkings'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')

    gamelist = []
    gamecards = soup.findAll("div", {"class": "game-card-teams"})
    for x in gamecards:
        twoteams = x.findAll("span", {"class": "team-nameplate-mascot"})
        roadteam = convert_name(twoteams[0].text)
        hometeam = convert_name(twoteams[1].text)
        gamekey = "{}@{}".format(roadteam,hometeam)
        gamelist.append(gamekey)

    matchupsdf = pd.DataFrame()
    for game in gamelist:
        roadteam = game.split("@")[0]
        hometeam = game.split("@")[1]
        thisdf1 = pd.DataFrame({"Team": roadteam, "Opp": hometeam, "RoadTeam": roadteam, "HomeTeam": hometeam},index=[0])
        thisdf2 = pd.DataFrame({"Team": hometeam, "Opp": roadteam, "RoadTeam": roadteam, "HomeTeam": hometeam},index=[0])
        matchupsdf = pd.concat([matchupsdf,thisdf1,thisdf2])
        
    oppdict = dict(zip(matchupsdf.Team,matchupsdf.Opp))
    hometeamdict = dict(zip(matchupsdf.Team,matchupsdf.HomeTeam))
    roadteamdict = dict(zip(matchupsdf.Team,matchupsdf.RoadTeam))

    disabled_span_list = []
    for span in soup.findAll("span", {"class": "player-nameplate disabled"}):
        for a in span.findAll("a"):
            disabled_span_list.append(a.text)

    spdata = pd.DataFrame()
    for div in soup.findAll("span", {"class": "player-nameplate", "data-position": "SP"}):
        if "TBD" in str(div):
            playername = "TBD"
            pos = "SP"
            sal = 0
        else:
            for a in div.findAll('a', {'class': 'player-nameplate-name'}):
                playername = a.text.strip()

            strdiv = str(div)
            pos = strdiv[strdiv.find("data-position")+15:strdiv.find("data-salary")-2]
            sal = strdiv[strdiv.find("data-salary")+13:strdiv.find("<div class = 'player-nameplate-info'>")-3]
        try:
            ownership = strdiv[strdiv.find('<span class="small muted" data-auth="502">') + 42:strdiv.find('%')]
            ownership = ownership.replace("</span>", "")
            ownership = ownership.replace("</span", "")
            ownership = ownership.replace("</div>", "")
            ownership = ownership.replace(" ", "")
        except:
            ownership = np.nan

        thisspdata = pd.DataFrame([[playername, sal, ownership]], columns = ["Player", "Salary", "Ownership"])
        spdata = pd.concat([spdata, thisspdata])

    spdata['Player'] = spdata['Player'].replace('Luis Ortiz', 'Luis L. Ortiz')
    spdata['Player'] = spdata['Player'].replace('Mike King', 'Michael King')
    spdata['Player'] = spdata['Player'].replace('Robert Zastryzny', 'Rob Zastryzny')

    spdata2 = pd.merge(spdata, PID[["Name", "Team"]], left_on = ["Player"], right_on = ["Name"], how = "left").rename(columns = {"Team": "PitcherTeam"})
    spdata3 = pd.merge(spdata2, matchupsdf[["Team", "Opp"]], left_on = ["PitcherTeam"], right_on = ["Team"], how = "left").drop(columns = ["Team"])

    append_suffix_to_duplicates(spdata3, 'PitcherTeam')
    append_suffix_to_duplicates(spdata3, 'Opp')

    opp_spname_dict = dict(zip(spdata3.Opp, spdata3.Player))
    opp_spsal_dict = dict(zip(spdata3.Opp, spdata.Salary))
    opp_spown_dict = dict(zip(spdata3.Opp, spdata3.Ownership))

    ludf = pd.DataFrame()
    
    for li in soup.findAll("li", {"class": "lineup-card-player"}):
        for a in li.findAll("a", {"class": ["player-nameplate-name", "player-nameplate disabled"]}):
            playername = a.text

        listring = str(li)
        for span in li.find("span", {"class": "small"}):
            luspot = span.text
            luspot = luspot.replace("\n", "")
            luspot = luspot.strip()
            luspot = int(luspot)
        pos = listring[listring.find("data-position")+15:listring.find("data-salary")-2]
        sal = listring[listring.find("data-salary")+13:listring.find("<span class='small'>")-3]
        ownership = ownership.replace("</span>", "")
        ownership = ownership.replace("</span", "")
        ownership = ownership.replace("</li", "")
        ownership = ownership.replace("</div>", "")
        ownership = ownership.replace(" ", "")

        try:
            sal = int(sal)
        except:
            sal = 0
        thisludf = pd.DataFrame([[playername, luspot, sal, ownership]], columns = ["Player", "Spot", "Sal", "Ownership"])
        ludf = pd.concat([ludf, thisludf])

    ludf2 = pd.merge(ludf, BID[["Name", "Team"]], left_on = ["Player"], right_on = ["Name"], how = "left").rename(columns = {"Team": "BatterTeam"})
    ludf2['BatterTeam'] = ludf2['BatterTeam'].fillna(method='ffill')
    ludf2['HomeTeam'] = ludf2['BatterTeam'].map(hometeamdict)
    ludf2['RoadTeam'] = ludf2['BatterTeam'].map(roadteamdict)

    ludf2_teamlist = list(ludf2["BatterTeam"])

    dhteams = []
    for x in ludf2_teamlist:
        if ludf2_teamlist.count(x) > 11:
            if x in dhteams:
                pass
            else:
                dhteams.append(x)

    extract_dh = ludf2[ludf2["BatterTeam"].isin(dhteams)]
    new_ludf2 = ludf2[~ludf2["BatterTeam"].isin(dhteams)]

    new_team_list = []
    new_home_list = []
    new_road_list = []
    runcounter = 0

    for x, home, road in zip(extract_dh["BatterTeam"].astype(str), 
                         extract_dh["HomeTeam"].astype(str), 
                         extract_dh["RoadTeam"].astype(str)):
        if runcounter < 18:
            new_team_list.append(x)
            new_home_list.append(home)
            new_road_list.append(road)
            runcounter += 1
        else:
            new_team_list.append(x + "2")
            new_home_list.append(home + "2")
            new_road_list.append(road + "2")
            runcounter += 1

    extract_dh["BatterTeam"] = new_team_list
    extract_dh["HomeTeam"] = new_home_list
    extract_dh["RoadTeam"] = new_road_list

    ludf2 = pd.concat([extract_dh, new_ludf2])
    ludf2["Opp"] = ludf2["BatterTeam"].map(oppdict)
    ludf2['SP'] = ludf2['BatterTeam'].map(opp_spname_dict)
    ludf2['SPSal'] = ludf2['BatterTeam'].map(opp_spsal_dict)
    ludf2['SPOwnership'] = ludf2['BatterTeam'].map(opp_spown_dict)
    ludf2['Date'] = todaysdate
    ludf2['Time'] = np.nan

    ludf3 = ludf2[['BatterTeam','RoadTeam','HomeTeam','Time','Spot','Player','Sal','Ownership','Date', "SP"]]

    dkdata = ludf3.copy()

    try:
        checknan = dkdata[["BatterTeam", "SP"]]
        getnans = checknan[["SP"].isna()]
        if len(getnans) == 0:
            nonans = 1
            nanmapdict = {}
        else:
            nonans = 0
            getnans["SP"] = disabled_span_list
            nanmapdict = dict(zip(getnans.Team, getnans.SP))
    except:
        pass

    try:
        dkdata["SP"] = np.where(dkdata["SP"].isna(), dkdata["BatterTeam"].map(nanmapdict), dkdata["SP"])
    except:
        pass
    
    for i in range(1, len(dkdata) - 1):
        if dkdata.loc[i, 'BatterTeam'] != dkdata.loc[i-1, 'BatterTeam']:
            if dkdata.loc[i, 'BatterTeam'] != dkdata.loc[i+1, 'BatterTeam']:
                dkdata.loc[i, 'BatterTeam'] = np.nan
                dkdata.loc[i, 'HomeTeam'] = np.nan
                dkdata.loc[i, 'RoadTeam'] = np.nan
                dkdata.loc[i, 'SP'] = np.nan

    
    dkdata[["BatterTeam", "RoadTeam", "HomeTeam"]] = dkdata[["BatterTeam", "RoadTeam", "HomeTeam"]].fillna(method='ffill')
    dkdata = dkdata.drop_duplicates(subset = ["BatterTeam", "SP"], keep = "first")
    dkdata = dkdata.drop(columns = ["Time", "Sal", "Ownership"])

    dkdata['BatterTeam'] = dkdata['BatterTeam'].replace('ARI', 'AZ')
    dkdata['RoadTeam'] = dkdata['RoadTeam'].replace('ARI', 'AZ')
    dkdata['HomeTeam'] = dkdata['HomeTeam'].replace('ARI', 'AZ')

    dkdata['Date'] = pd.to_datetime(dkdata['Date'])
    dkdata['Date'] = dkdata['Date'].dt.strftime('%Y-%m-%d')
    dkdata = dkdata.set_index("Date")
    dkdata = dkdata[["BatterTeam", "RoadTeam", "HomeTeam", "SP"]]

    return(dkdata)

## Loads regular season data from 2021-23 to train on

In [51]:
#statcast(start_dt = "2021-04-01", end_dt = "2021-10-03")
#statcast(start_dt = "2022-04-07", end_dt = "2022-10-02")
#statcast(start_dt = "2023-03-30", end_dt = "2023-10-01")

savant2021 = pl.read_parquet("~/Desktop/Desktop - Cameron MacBook Pro/Random-Projects/MLB/Data/savant2021.parquet")
savant2022 = pl.read_parquet("~/Desktop/Desktop - Cameron MacBook Pro/Random-Projects/MLB/Data/savant2022.parquet")
savant2023 = pl.read_parquet("~/Desktop/Desktop - Cameron MacBook Pro/Random-Projects/MLB/Data/savant2023.parquet")

In [52]:
# Concatenate DataFrames
combined1 = pl.concat([savant2021, savant2022, savant2023])

# Convert game_date using flexible datetime parsing
combined1 = combined1.with_columns(pl.col('game_date').str.to_datetime().alias('game_date'))

# Create BatterTeam and PitcherTeam columns using conditional logic
combined1 = combined1.with_columns([pl.when(pl.col('inning_topbot') == 'Top').then(pl.col('away_team')).otherwise(pl.col('home_team')).alias('BatterTeam'),
    pl.when(pl.col('inning_topbot') == 'Top').then(pl.col('home_team')).otherwise(pl.col('away_team')).alias('PitcherTeam')])

# Calculate runs scored
combined1 = combined1.with_columns([
    (pl.col('post_away_score') - pl.col('away_score')).alias('AwayRunsScored'),
    (pl.col('post_home_score') - pl.col('home_score')).alias('HomeRunsScored')])

# Apply custom functions to player_name
combined1 = combined1.with_columns(pl.col('player_name').map_elements(flip_names, return_dtype=pl.Utf8).alias('player_name'))
combined1 = combined1.with_columns(pl.col('player_name').map_elements(replace_special_chars, return_dtype=pl.Utf8).alias('player_name'))

## Creating a DF where it only loads in the stats of starting pitchers

In [54]:
# Define groupby columns
groupby_cols = ['game_date', 'BatterTeam', 'away_team', 'home_team']

# Select and sort columns (equivalent to your combined2 creation)
combined2 = combined1.select([
    'game_date', 'home_team', 'away_team', 'inning', 'inning_topbot', 'at_bat_number', 'pitch_number',
    'BatterTeam', 'MLBNAME', 'events', 'description', 'bb_type', 'estimated_ba_using_speedangle',
    'estimated_woba_using_speedangle', 'woba_value', 'p_throws', 'PitcherTeam', 'player_name',
    'delta_run_exp', 'AwayRunsScored', 'HomeRunsScored']).sort([
    'game_date', 'home_team', 'away_team', 'inning_topbot', 'at_bat_number', 'pitch_number'], descending=[False, False, False, True, False, False])

# Get the first (starter) player_name for each group
starter_names = combined2.group_by(groupby_cols).agg(pl.col('player_name').first().alias('starter_name'))

# Join back to combined2 to get starter information
joined = combined2.join(starter_names, on=groupby_cols, how='left')

# Filter to keep only rows where player_name matches the starter
df_starters_only = joined.filter(pl.col('player_name') == pl.col('starter_name')).drop('starter_name')

# Define count_outs function for Polars (if needed later)
def count_outs(events_column):
    single_outs = ['other_out', 'strikeout', 'field_out', 'force_out', 'fielders_choice', 
                   'fielders_choice_out', 'sac_fly', 'sac_bunt', 'caught_stealing_2b', 
                   'caught_stealing_3b', 'caught_stealing_home', 'pickoff_caught_stealing_2b',  
                   'pickoff_caught_stealing_3b', 'pickoff_caught_stealing_home']
    double_outs = ['double_play', 'strikeout_double_play', 'grounded_into_double_play', 'sac_fly_double_play']
    triple_outs = ['triple_play']
    
    return (events_column.is_in(single_outs).sum() + 
        2 * events_column.is_in(double_outs).sum() + 
        3 * events_column.is_in(triple_outs).sum())

## Grouping on a pitch level for pitchers

In [55]:
# Perform the groupby aggregation in Polars
Train1 = df_starters_only.group_by([
    "game_date", "BatterTeam", "away_team", "home_team", "inning", "at_bat_number", "MLBNAME", "player_name", "p_throws"]).agg([pl.count("pitch_number").alias("Pitches"),pl.col("events").filter(pl.col("events").is_in(['other_out', 'single', 'double', 'triple', 'home_run', 'strikeout', 'field_out', 'field_error',                          'fielders_choice', 'double_play', 'fielders_choice_out', 'strikeout_double_play', 'triple_play', 'grounded_into_double_play'])).count().alias("PA"),
    pl.col("events").map_elements(count_outs, return_dtype=pl.Int64).alias("Outs"),
    pl.col("events").filter(pl.col("events").is_in(['single', 'double', 'triple', 'home_run'])).count().alias("Hit"),
    pl.col("bb_type").filter(pl.col("bb_type") == 'fly_ball').count().alias("FB"),
    pl.col("events").filter(pl.col("events") == 'home_run').count().alias("HR"),
    pl.col("events").filter(pl.col("events") == 'hit_by_pitch').count().alias("HBP"),
    pl.col("description").filter(pl.col("description").is_in(["ball", "hit_by_pitch", "blocked_ball"])).count().alias("Balls"),
    pl.col("events").filter(pl.col("events") == 'walk').count().alias("BB"),
    pl.col("description").filter(pl.col("description") == 'called_strike').count().alias("CS"),
    pl.col("description").filter(pl.col("description").is_in(["swinging_strike", "swinging_strike_blocked", "foul_tip"])).count().alias("Whiff"),
    pl.col("description").filter(pl.col("description").is_in(["called_strike", "swinging_strike", "foul", "swinging_strike_blocked", "foul_tip"])).count().alias("Strikes"),
    pl.col("events").filter(pl.col("events") == 'strikeout').count().alias("K"),
    pl.col("estimated_ba_using_speedangle").mean().alias("xBA"),
    pl.col("estimated_woba_using_speedangle").mean().alias("xwOBA"),
    pl.col("woba_value").mean().alias("wOBA"),
    pl.col("delta_run_exp").mean().alias("RunExp"),
    pl.col("AwayRunsScored").sum().alias("AwayRunsScored"),
    pl.col("HomeRunsScored").sum().alias("HomeRunsScored")]).fill_null(0)

## Grouping on an at bat level

In [66]:
# Group by and aggregate
Train2 = Train1.group_by([
    "game_date", "BatterTeam", "away_team", "home_team", "player_name", "p_throws"]).agg([
    pl.col("Pitches").sum().alias("Pitches"),
    pl.count("at_bat_number").alias("AB"),
    pl.col("PA").sum().alias("PA"),
    pl.col("Outs").sum().alias("Outs"),
    pl.col("Hit").sum().alias("Hits"),
    pl.col("FB").sum().alias("FB"),
    pl.col("HR").sum().alias("HR"),
    pl.col("HBP").sum().alias("HBP"),
    pl.col("Balls").sum().alias("Balls"),
    pl.col("BB").sum().alias("BB"),
    pl.col("CS").sum().alias("CS"),
    pl.col("Whiff").sum().alias("Whiff"),
    pl.col("Strikes").sum().alias("Strikes"),
    pl.col("K").sum().alias("K"),
    pl.col("xBA").mean().alias("xBA"),
    pl.col("xwOBA").mean().alias("xwOBA"),
    pl.col("wOBA").mean().alias("wOBA"),
    pl.col("RunExp").mean().alias("RunExp"),
    pl.col("AwayRunsScored").sum().alias("AwayRunsScored"),
    pl.col("HomeRunsScored").sum().alias("HomeRunsScored")]).fill_null(0)

# Create the RA column using conditional logic
Train2 = Train2.with_columns(
    pl.when((pl.col('HomeRunsScored') > 0) & (pl.col('BatterTeam') == pl.col('home_team')))
      .then(pl.col('HomeRunsScored'))
      .when((pl.col('AwayRunsScored') > 0) & (pl.col('BatterTeam') == pl.col('away_team')))
      .then(pl.col('AwayRunsScored'))
      .otherwise(0)
      .alias('RA'))

# Create RA/9 column
Train2 = Train2.with_columns((27 * pl.col('RA') / pl.col('Outs')).alias('RA/9'))

# Drop AwayRunsScored and HomeRunsScored columns
Train2 = Train2.drop(['AwayRunsScored', 'HomeRunsScored'])

## Calculating loads of pitcher rate stats

In [67]:
# Calculate league-wide statistics
lgHR = combined1.filter(pl.col("events") == "home_run").height
lgFB = combined1.filter(pl.col("bb_type") == "fly_ball").height

# Add calculated columns to Train2
Train2 = Train2.with_columns([
    # FIP calculation
    ((13 * pl.col('HR') + 3 * (pl.col('BB') + pl.col('HBP')) - 2 * pl.col('K')) / (pl.col('Outs') / 3) + 3.137).alias('FIP'),
    
    # xFIP calculation
    ((13 * (pl.col('FB') * (lgHR / lgFB * 0.58)) + 3 * (pl.col('BB') + pl.col('HBP')) - 2 * pl.col('K')) / (pl.col('Outs') / 3) + 3.137).alias('xFIP'),
    
    # Percentage calculations
    (100 * (pl.col('K') / pl.col('AB'))).round(2).alias('K%'),
    (100 * (pl.col('BB') / pl.col('AB'))).round(2).alias('BB%'),
    (100 * (pl.col('Balls') / pl.col('Pitches'))).round(2).alias('Ball%'),
    (100 * (pl.col('Strikes') / pl.col('Pitches'))).round(2).alias('Strike%'),
    (100 * (pl.col('CS') / pl.col('Pitches'))).round(2).alias('CS%'),
    (100 * (pl.col('Whiff') / pl.col('Pitches'))).round(2).alias('Whiff%'),
    
    # CSW calculation
    (pl.col('CS') + pl.col('Whiff')).alias('CSW')])

# Add K-BB% and CSW% calculations
Train2 = Train2.with_columns([
    (pl.col('K%') - pl.col('BB%')).alias('K-BB%'),
    (100 * (pl.col('CSW') / pl.col('Pitches'))).round(2).alias('CSW%')])

# Drop unnecessary columns
Train2 = Train2.drop(['CSW'])

## Adding rolling averages for the past 5 and 10 games

In [68]:
window_size5 = 5
window_size10 = 10
window_size20 = 20

# Sort Train2 by player_name and game_date for rolling
Train2 = Train2.sort(['player_name', 'game_date'])

# Rolling calculations using .rolling_mean(window, min_samples=1).over('player_name')
Train2 = Train2.with_columns([
    pl.col('Pitches').rolling_mean(window_size5, min_samples=1).over('player_name').alias('Pitches5'),
    pl.col('Outs').rolling_mean(window_size5, min_samples=1).over('player_name').alias('Outs5'),
    pl.col('Outs').rolling_mean(window_size10, min_samples=1).over('player_name').alias('Outs10'),
    pl.col('xBA').rolling_mean(window_size5, min_samples=1).over('player_name').alias('xBA5'),
    pl.col('xBA').rolling_mean(window_size10, min_samples=1).over('player_name').alias('xBA10'),
    pl.col('xwOBA').rolling_mean(window_size5, min_samples=1).over('player_name').alias('xwOBA5'),
    pl.col('xwOBA').rolling_mean(window_size10, min_samples=1).over('player_name').alias('xwOBA10'),
    pl.col('wOBA').rolling_mean(window_size5, min_samples=1).over('player_name').alias('wOBA5'),
    pl.col('wOBA').rolling_mean(window_size10, min_samples=1).over('player_name').alias('wOBA10'),
    pl.col('RA/9').rolling_mean(window_size5, min_samples=1).over('player_name').alias('RA5'),
    pl.col('RA/9').rolling_mean(window_size10, min_samples=1).over('player_name').alias('RA10'),
    pl.col('FIP').rolling_mean(window_size5, min_samples=1).over('player_name').alias('FIP5'),
    pl.col('FIP').rolling_mean(window_size10, min_samples=1).over('player_name').alias('FIP10'),
    pl.col('xFIP').rolling_mean(window_size5, min_samples=1).over('player_name').alias('xFIP5'),
    pl.col('xFIP').rolling_mean(window_size10, min_samples=1).over('player_name').alias('xFIP10'),
    pl.col('K%').rolling_mean(window_size5, min_samples=1).over('player_name').alias('K%5'),
    pl.col('K%').rolling_mean(window_size10, min_samples=1).over('player_name').alias('K%10'),
    pl.col('BB%').rolling_mean(window_size5, min_samples=1).over('player_name').alias('BB%5'),
    pl.col('BB%').rolling_mean(window_size10, min_samples=1).over('player_name').alias('BB%10'),
    pl.col('K-BB%').rolling_mean(window_size5, min_samples=1).over('player_name').alias('K-BB%5'),
    pl.col('K-BB%').rolling_mean(window_size10, min_samples=1).over('player_name').alias('K-BB%10'),
    pl.col('Ball%').rolling_mean(window_size5, min_samples=1).over('player_name').alias('Ball%5'),
    pl.col('Strike%').rolling_mean(window_size5, min_samples=1).over('player_name').alias('Strike%5'),
    pl.col('CS%').rolling_mean(window_size5, min_samples=1).over('player_name').alias('CS%5'),
    pl.col('CS%').rolling_mean(window_size10, min_samples=1).over('player_name').alias('CS%10'),
    pl.col('Whiff%').rolling_mean(window_size5, min_samples=1).over('player_name').alias('Whiff%5'),
    pl.col('Whiff%').rolling_mean(window_size10, min_samples=1).over('player_name').alias('Whiff%10'),
    pl.col('CSW%').rolling_mean(window_size5, min_samples=1).over('player_name').alias('CSW%5'),
    pl.col('CSW%').rolling_mean(window_size10, min_samples=1).over('player_name').alias('CSW%10')])

# Filter out rows where Pitches5 < 40
Train2 = Train2.filter(pl.col('Pitches5') >= 40)

# Drop specified columns
Train3 = Train2.drop([
    "FB", "Balls", "HBP", "CS", "Whiff", "Strikes", 
    'Ball%', 'Strike%', 'CS%', 'Whiff%', 'CSW%', "RA/9"])

# Rename columns
Train3 = Train3.rename({
    'away_team': 'RoadTeam', 
    'home_team': 'HomeTeam', 
    "player_name": "SP"})

## Grouping on a pitch level for batters

In [86]:
# Define the count_outs function for out events
out_events = ['other_out', 'field_out', 'field_error', 'fielders_choice', 'fielders_choice_out', 'double_play', 'strikeout_double_play', 'triple_play', 'grounded_into_double_play']

# Polars groupby aggregation
BatterTrain1 = combined2.group_by([
    "game_date", "BatterTeam", "away_team", "home_team", "inning", "at_bat_number", "MLBNAME", "player_name", "p_throws"]).agg([
    pl.col("pitch_number").len().alias("Pitches"),
    pl.col("events").filter(pl.col("events").is_in(['other_out', 'single', 'double', 'triple', 'home_run', 'strikeout', 'field_out', 'field_error', 'fielders_choice', 'double_play', 'fielders_choice_out', 'strikeout_double_play', 'triple_play', 'grounded_into_double_play'])).len().alias("PA"),
    pl.col("events").filter(pl.col("events").is_in(out_events)).len().alias("Outs"),
    pl.col("bb_type").filter(pl.col("bb_type") == 'fly_ball').len().alias("FB"),
    pl.col("events").filter(pl.col("events") == 'home_run').len().alias("HR"),
    pl.col("events").filter(pl.col("events") == 'hit_by_pitch').len().alias("HBP"),
    pl.col("description").filter(pl.col("description").is_in(["ball", "hit_by_pitch", "blocked_ball"])).len().alias("Balls"),
    pl.col("events").filter(pl.col("events") == 'walk').len().alias("BB"),
    pl.col("description").filter(pl.col("description") == 'called_strike').len().alias("CS"),
    pl.col("description").filter(pl.col("description").is_in(["swinging_strike", "swinging_strike_blocked", "foul_tip"])).len().alias("Whiff"),
    pl.col("description").filter(pl.col("description").is_in(["called_strike", "swinging_strike", "foul", "swinging_strike_blocked", "foul_tip"])).len().alias("Strikes"),
    pl.col("events").filter(pl.col("events") == 'strikeout').len().alias("K"),
    pl.col("estimated_ba_using_speedangle").mean().alias("xBA"),
    pl.col("estimated_woba_using_speedangle").mean().alias("xwOBA"),
    pl.col("woba_value").mean().alias("wOBA"),
    pl.col("delta_run_exp").mean().alias("RunExp"),
    pl.col("AwayRunsScored").sum().alias("AwayRunsScored"),
    pl.col("HomeRunsScored").sum().alias("HomeRunsScored")]).fill_null(0)

## Grouping on an at bat level

In [95]:
# First groupby aggregation
BatterTrain2 = BatterTrain1.group_by([
    "game_date", "BatterTeam", "away_team", "home_team", "p_throws"]).agg([
    pl.col("Pitches").sum().alias("Pitches"),
    pl.col("at_bat_number").len().alias("AB"),
    pl.col("PA").sum().alias("PA"),
    pl.col("Outs").sum().alias("Outs"),
    pl.col("FB").sum().alias("FB"),
    pl.col("HR").sum().alias("HR"),
    pl.col("HBP").sum().alias("HBP"),
    pl.col("Balls").sum().alias("Balls"),
    pl.col("BB").sum().alias("BB"),
    pl.col("CS").sum().alias("CS"),
    pl.col("Whiff").sum().alias("Whiff"),
    pl.col("Strikes").sum().alias("Strikes"),
    pl.col("K").sum().alias("K"),
    pl.col("xBA").mean().alias("xBA"),
    pl.col("xwOBA").mean().alias("xwOBA"),
    pl.col("wOBA").mean().alias("wOBA"),
    pl.col("RunExp").mean().alias("RunExp"),
    pl.col("AwayRunsScored").sum().alias("AwayRunsScored"),
    pl.col("HomeRunsScored").sum().alias("HomeRunsScored")]).fill_null(0)

# Add calculated columns and drop unwanted columns
BatterTrain2 = BatterTrain2.with_columns([
    # Create RA column using conditional logic
    pl.when((pl.col("HomeRunsScored") > 0) & (pl.col("BatterTeam") == pl.col("home_team"))).then(pl.col("HomeRunsScored")).when(
        (pl.col("AwayRunsScored") > 0) & (pl.col("BatterTeam") == pl.col("away_team"))).then(pl.col("AwayRunsScored")).otherwise(0).alias("RA"),]).with_columns([
    # Create R/9 column
    (27 * pl.col("RA") / pl.col("Outs")).alias("R/9")]).drop(["AwayRunsScored", "HomeRunsScored", "RA"])

In [96]:
# Add calculated columns using with_columns
BatterTrain2 = BatterTrain2.with_columns([
    # FIP calculation
    ((13 * pl.col('HR') + 3 * (pl.col('BB') + pl.col('HBP')) - 2 * pl.col('K')) / 
     (pl.col('Outs') / 3) + 3.137).alias('FIP'),
    
    # xFIP calculation (assuming lgHR and lgFB are defined variables)
    ((13 * (pl.col('FB') * (lgHR/lgFB * 0.58)) + 3 * (pl.col('BB') + pl.col('HBP')) - 2 * pl.col('K')) / 
     (pl.col('Outs') / 3) + 3.137).alias('xFIP'),
    
    # Percentage calculations with rounding
    ((pl.col('K') / pl.col('AB')) * 100).round(2).alias('K%'),
    ((pl.col('BB') / pl.col('AB')) * 100).round(2).alias('BB%'),
    ((pl.col('Balls') / pl.col('Pitches')) * 100).round(2).alias('Ball%'),
    ((pl.col('Strikes') / pl.col('Pitches')) * 100).round(2).alias('Strike%'),
    ((pl.col('CS') / pl.col('Pitches')) * 100).round(2).alias('CS%'),
    ((pl.col('Whiff') / pl.col('Pitches')) * 100).round(2).alias('Whiff%'),
    
    # CSW calculation (temporary column)
    (pl.col('CS') + pl.col('Whiff')).alias('CSW')]).with_columns([
    # K-BB% calculation (depends on K% and BB% from previous step)
    (pl.col('K%') - pl.col('BB%')).alias('K-BB%'),
    
    # CSW% calculation
    ((pl.col('CSW') / pl.col('Pitches')) * 100).round(2).alias('CSW%')]).drop(['CSW'])

In [98]:
Train3 = BatterTrain2.with_columns([
    # Rolling 5 and 10 game expected batting averages
    pl.col("xBA").rolling_mean(window_size=window_size5, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bxBA5"),
    pl.col("xBA").rolling_mean(window_size=window_size10, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bxBA10"),
    
    # Rolling 5 and 10 game expected wOBA averages
    pl.col("xwOBA").rolling_mean(window_size=window_size5, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bxwOBA5"),
    pl.col("xwOBA").rolling_mean(window_size=window_size10, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bxwOBA10"),
    
    # Rolling 5 and 10 game wOBA averages
    pl.col("wOBA").rolling_mean(window_size=window_size5, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bwOBA5"),
    pl.col("wOBA").rolling_mean(window_size=window_size10, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bwOBA10"),
    
    # Rolling 5 and 10 game RA averages
    pl.col("R/9").rolling_mean(window_size=window_size5, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bRS5"),
    pl.col("R/9").rolling_mean(window_size=window_size10, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bRS10"),
    
    # Rolling 5 and 10 game FIP averages
    pl.col("FIP").rolling_mean(window_size=window_size5, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bFIP5"),
    pl.col("FIP").rolling_mean(window_size=window_size10, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bFIP10"),
    
    # Rolling 5 and 10 game xFIP averages
    pl.col("xFIP").rolling_mean(window_size=window_size5, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bxFIP5"),
    pl.col("xFIP").rolling_mean(window_size=window_size10, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bxFIP10"),
    
    # Rolling 5 and 10 game K% averages
    pl.col("K%").rolling_mean(window_size=window_size5, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bK%5"),
    pl.col("K%").rolling_mean(window_size=window_size10, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bK%10"),
    
    # Rolling 5 and 10 game BB% averages
    pl.col("BB%").rolling_mean(window_size=window_size5, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bBB%5"),
    pl.col("BB%").rolling_mean(window_size=window_size10, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bBB%10"),
    
    # Rolling 5 and 10 game K-BB% averages
    pl.col("K-BB%").rolling_mean(window_size=window_size5, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bK-BB%5"),
    pl.col("K-BB%").rolling_mean(window_size=window_size10, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bK-BB%10"),
    
    # Rolling 5 game Ball% averages
    pl.col("Ball%").rolling_mean(window_size=window_size5, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bBall%5"),
    
    # Rolling 5 game Strike% averages
    pl.col("Strike%").rolling_mean(window_size=window_size5, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bStrike%5"),
    
    # Rolling 5 and 10 game Called Strike% averages
    pl.col("CS%").rolling_mean(window_size=window_size5, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bCS%5"),
    pl.col("CS%").rolling_mean(window_size=window_size10, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bCS%10"),
    
    # Rolling 5 and 10 game Whiff% averages
    pl.col("Whiff%").rolling_mean(window_size=window_size5, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bWhiff%5"),
    pl.col("Whiff%").rolling_mean(window_size=window_size10, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bWhiff%10"),
    
    # Rolling 5 and 10 game Called Strike plus Whiff% averages
    pl.col("CSW%").rolling_mean(window_size=window_size5, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bCSW%5"),
    pl.col("CSW%").rolling_mean(window_size=window_size10, min_samples=1).over(["BatterTeam", "p_throws"]).alias("bCSW%10")
])

# Replace infinite values with 5
Train3 = Train3.with_columns([
    pl.when(pl.col(col).is_infinite())
    .then(5)
    .otherwise(pl.col(col))
    .alias(col)
    for col in Train3.columns if col.startswith('b')])

## Loads in today's data

In [100]:
TodaysData = getDKData2024()
TodaysData

/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_65307/2850615655.py:101: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ludf2['BatterTeam'] = ludf2['BatterTeam'].fillna(method='ffill')
/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_65307/2850615655.py:180: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dkdata[["BatterTeam", "RoadTeam", "HomeTeam"]] = dkdata[["BatterTeam", "RoadTeam", "HomeTeam"]].fillna(method='ffill')


,BatterTeam,RoadTeam,HomeTeam,SP
Date,,,,
2025-06-22,BAL,BAL,NYY,Will Warren
2025-06-22,NYY,BAL,NYY,Dean Kremer
2025-06-22,DET,DET,TB,Zack Littell
2025-06-22,TB,DET,TB,Casey Mize
2025-06-22,TEX,TEX,PIT,Bailey Falter
2025-06-22,PIT,TEX,PIT,Jack Leiter
2025-06-22,CWS,CWS,TOR,Chris Bassitt
2025-06-22,TOR,CWS,TOR,Adrian Houser
2025-06-22,ATL,ATL,MIA,Sandy Alcantara


In [ ]:
eastern_time = datetime.datetime.now(timezone.utc).astimezone(timezone(datetime.timedelta(hours=-5)))
savant2024 = statcast(start_dt = "2024-03-28", end_dt = eastern_time.strftime("%Y-%m-%d"))
savant2024['game_date'] = pd.to_datetime(savant2024['game_date'])
savant2024['game_date'] = savant2024['game_date'].dt.strftime('%Y-%m-%d')
savant2024['BatterTeam'] = np.where(savant2024['inning_topbot'] == 'Top', savant2024['away_team'], savant2024['home_team'])
savant2024['PitcherTeam'] = np.where(savant2024['inning_topbot'] == 'Top', savant2024['home_team'], savant2024['away_team'])
savant2024 = pd.merge(savant2024, ID[["MLBID", "MLBNAME"]], left_on = 'batter', right_on = 'MLBID', how = 'left')
savant2024.dropna(subset=['MLBNAME'], inplace=True)
savant2024 = savant2024.drop_duplicates(subset = ["pitch_type", "game_date", "release_speed", "release_pos_x", "release_pos_z", "player_name"], keep='first')
savant2024["player_name"] = savant2024["player_name"].apply(flip_names)
savant2024['AwayRunsScored'] = savant2024['post_away_score'] - savant2024['away_score']
savant2024['HomeRunsScored'] = savant2024['post_home_score'] - savant2024['home_score']
savant2024 = savant2024[["game_date", "home_team", "away_team", "inning", "inning_topbot", "at_bat_number", "pitch_number", "pitch_type", "BatterTeam", "MLBNAME", "balls", "strikes", "outs_when_up", "events", "description", "bb_type", "hit_distance_sc", "launch_speed", "launch_angle", "estimated_ba_using_speedangle", "estimated_woba_using_speedangle", "woba_value", "p_throws", "PitcherTeam", "player_name", "delta_home_win_exp", "delta_run_exp", "away_score", "home_score", "AwayRunsScored", "HomeRunsScored"]].sort_values(by = ["game_date", "home_team", "away_team", "inning_topbot", "at_bat_number", "pitch_number"], ascending=[True, True, True, False, True, True])
savant2024 = savant2024.groupby(groupby_cols).apply(keep_starter).reset_index(drop = True)
savant2024["player_name"] = savant2024["player_name"].apply(replace_special_chars)

## Does the same grouping as the training data at the various levels

In [ ]:
Season1 = savant2024.groupby(["game_date", "BatterTeam", "away_team", "home_team", "inning", "at_bat_number", "MLBNAME", "player_name", "p_throws"]).agg(
    Pitches = ("pitch_number", "size"),
    PA = ('events', lambda x: (x.isin(['other_out', 'single', 'double', 'triple', 'home_run', 'strikeout', 'field_out', 'field_error', 'fielders_choice', 'double_play', 'fielders_choice_out', 'strikeout_double_play', 'triple_play', 'grounded_into_double_play'])).sum()),
    Outs = ('events', count_outs),
    Hit = ('events', lambda x: (x.isin(['single', 'double', 'triple', 'home_run'])).sum()),
    FB = ('bb_type', lambda x: (x == 'fly_ball').sum()),
    HR = ('events', lambda x: (x == 'home_run').sum()),
    HBP = ('events', lambda x: (x == 'hit_by_pitch').sum()),
    Balls = ('description', lambda x: (x.isin(["ball", "hit_by_pitch", "blocked_ball"])).sum()),
    BB = ('events', lambda x: (x == 'walk').sum()),
    CS = ('description', lambda x: (x == 'called_strike').sum()),
    Whiff = ('description', lambda x: (x.isin(["swinging_strike", "swinging_strike_blocked", "foul_tip"])).sum()),
    Strikes = ('description', lambda x: (x.isin(["called_strike", "swinging_strike", "foul", "swinging_strike_blocked", "foul_tip"])).sum()),
    K = ('events', lambda x: (x == 'strikeout').sum()),
    xBA = ("estimated_ba_using_speedangle", "mean"),
    xwOBA = ("estimated_woba_using_speedangle", "mean"),
    wOBA = ("woba_value", "mean"),
    RunExp = ("delta_run_exp", "mean"),
    AwayRunsScored = ("AwayRunsScored", "sum"),
    HomeRunsScored = ("HomeRunsScored", "sum")).reset_index().fillna(0)

In [ ]:
Season2 = Season1.groupby(["game_date", "BatterTeam", "away_team", "home_team", "player_name", "p_throws"]).agg(
    Pitches = ("Pitches", "sum"),
    AB = ("at_bat_number", "size"),
    PA = ("PA", "sum"),
    Outs = ("Outs", "sum"),
    Hits = ("Hit", "sum"),
    FB = ("FB", "sum"),
    HR = ("HR", "sum"),
    HBP = ("HBP", "sum"),
    Balls = ("Balls", "sum"),
    BB = ("BB", "sum"),
    CS = ("CS", "sum"),
    Whiff = ("Whiff", "sum"),
    Strikes = ("Strikes", "sum"),
    K = ("K", "sum"),
    xBA = ("xBA", "mean"),
    xwOBA = ("xwOBA", "mean"),
    wOBA = ("wOBA", "mean"),
    RunExp = ("RunExp", "mean"),
    AwayRunsScored = ("AwayRunsScored", "sum"),
    HomeRunsScored = ("HomeRunsScored", "sum")).reset_index().fillna(0)

Season2["IP"] = Season2["Outs"] / 3
Season2['RA'] = np.where((Season2['HomeRunsScored'] > 0) & (Season2['BatterTeam'] == Season2['home_team']), Season2["HomeRunsScored"],
                        np.where((Season2['AwayRunsScored'] > 0) & (Season2['BatterTeam'] == Season2['away_team']),Season2["AwayRunsScored"], 0))
Season2["RA/9"] = (27 * Season2["RA"] / Season2["Outs"])
Season2 = Season2.drop(columns = ["AwayRunsScored", "HomeRunsScored"])

In [ ]:
Season2['FIP'] = (13 * Season2['HR'] + 3 * (Season2['BB'] + Season2['HBP']) - 2 * Season2['K']) / (Season2['IP']) + 3.137
Season2['xFIP'] = (13 * (Season2['FB'] * (lgHR/lgFB * 0.58)) + 3 * (Season2['BB'] + Season2['HBP']) - 2 * Season2['K']) / (Season2['IP']) + 3.137

Season2['K%'] = round((Season2['K'] / Season2['AB']) * 100, 2)
Season2['BB%'] = round((Season2['BB'] / Season2['AB']) * 100, 2)
Season2['K-BB%'] = Season2["K%"] - Season2["BB%"]
Season2['Ball%'] = round((Season2['Balls'] / Season2['Pitches']) * 100, 2)
Season2['Strike%'] = round((Season2['Strikes'] / Season2['Pitches']) * 100, 2)
Season2['CS%'] = round((Season2['CS'] / Season2['Pitches']) * 100, 2)
Season2['Whiff%'] = round((Season2['Whiff'] / Season2['Pitches']) * 100, 2)
Season2["CSW"] = Season2["CS"] + Season2["Whiff"]
Season2['CSW%'] = round((Season2['CSW'] / Season2['Pitches']) * 100, 2)
Season2 = Season2.drop(columns=["CSW"])

In [ ]:
# Rolling 5 game pitch averages
Season2['Pitches5'] = Season2.groupby('player_name')['Pitches'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2 = Season2.drop(Season2[Season2['Pitches5'] < 40].index)
# Rolling 5 and 10 game outs averages
Season2['Outs5'] = Season2.groupby('player_name')['Outs'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['Outs10'] = Season2.groupby('player_name')['Outs'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game expected batting averages
Season2['xBA5'] = Season2.groupby('player_name')['xBA'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['xBA10'] = Season2.groupby('player_name')['xBA'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game expected wOBA averages
Season2['xwOBA5'] = Season2.groupby('player_name')['xwOBA'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['xwOBA10'] = Season2.groupby('player_name')['xwOBA'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game wOBA averages
Season2['wOBA5'] = Season2.groupby('player_name')['wOBA'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['wOBA10'] = Season2.groupby('player_name')['wOBA'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game RA averages
Season2['RA5'] = Season2.groupby('player_name')['RA/9'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['RA10'] = Season2.groupby('player_name')['RA/9'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game FIP averages
Season2['FIP5'] = Season2.groupby('player_name')['FIP'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['FIP10'] = Season2.groupby('player_name')['FIP'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game xFIP averages
Season2['xFIP5'] = Season2.groupby('player_name')['xFIP'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['xFIP10'] = Season2.groupby('player_name')['xFIP'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game K% averages
Season2['K%5'] = Season2.groupby('player_name')['K%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['K%10'] = Season2.groupby('player_name')['K%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game BB% averages
Season2['BB%5'] = Season2.groupby('player_name')['BB%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['BB%10'] = Season2.groupby('player_name')['BB%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game K-BB% averages
Season2['K-BB%5'] = Season2.groupby('player_name')['K-BB%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['K-BB%10'] = Season2.groupby('player_name')['K-BB%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 game Ball% averages
Season2['Ball%5'] = Season2.groupby('player_name')['Ball%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 game Strike% averages
Season2['Strike%5'] = Season2.groupby('player_name')['Strike%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game Called Strike% averages
Season2['CS%5'] = Season2.groupby('player_name')['CS%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['CS%10'] = Season2.groupby('player_name')['CS%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game Whiff% averages
Season2['Whiff%5'] = Season2.groupby('player_name')['Whiff%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['Whiff%10'] = Season2.groupby('player_name')['Whiff%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)
# Rolling 5 and 10 game Called Strike plus Whiff% averages
Season2['CSW%5'] = Season2.groupby('player_name')['CSW%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=0, drop=True)
Season2['CSW%10'] = Season2.groupby('player_name')['CSW%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=0, drop=True)

Season3 = Season2.drop(columns = ["FB", "Balls", "HBP", "CS", "Whiff", "Strikes", 'Ball%', 'Strike%', 'CS%', 'Whiff%', 'CSW%', "RA/9"])
Season3 = Season3.rename(columns={'away_team': 'RoadTeam', 'home_team': 'HomeTeam', "player_name": "SP"})

## Grouping everything to get season and rolling averages for pitchers

In [ ]:
Season4 = Season3.groupby(["SP", "p_throws"]).agg(
    Starts = ("IP", "size"),
    Pitches = ("Pitches", "mean"),
    AB = ("AB", "mean"),
    PA = ("PA", "mean"),
    Outs = ("Outs", "mean"),
    Hits = ("Hits", "mean"),
    HR = ("HR", "mean"),
    BB = ("BB", "mean"),
    K = ("K", "mean"),
    RA = ("RA", "mean"),
    xBA = ("xBA", "mean"),
    xwOBA = ("xwOBA", "mean"),
    wOBA = ("wOBA", "mean"),
    RunExp = ("RunExp", "mean"),
    FIP = ("FIP", "mean"),
    xFIP = ("xFIP", "mean"),
    Kpercent = ("K%", "mean"),
    BBpercent = ("BB%", "mean"),
    KminusBBpercent = ("K-BB%", "mean"),
    Pitches5 =  ("Pitches5", "last"),
    Outs5 =  ("Outs5", "last"),
    Outs10 = ("Outs10", "last"),
    xBA5 =  ("xBA5", "last"),
    xBA10 = ("xBA10", "last"),
    xwOBA5 =  ("xwOBA5", "last"),
    xwOBA10 = ("xwOBA10", "last"),
    wOBA5 =  ("wOBA5", "last"),
    wOBA10 = ("wOBA10", "last"),
    RA5 = ("RA5", "last"),
    RA10 = ("RA10", "last"),
    FIP5 = ("FIP5", "last"),
    FIP10 = ("FIP10", "last"),
    xFIP5 = ("xFIP5", "last"),
    xFIP10 = ("xFIP10", "last"),
    Kpercent5 = ("K%5", "last"),
    Kpercent10 = ("K%10", "last"),
    BBpercent5 = ("BB%5", "last"),
    BBpercent10 = ("BB%10", "last"),
    KminusBBpercent5 = ("K-BB%5", "last"),
    KminusBBpercent10 = ("K-BB%10", "last"),
    Ballpercent5 = ("Ball%5", "last"),
    Strikepercent5 = ("Strike%5", "last"),
    CSpercent5 = ("CS%5", "last"),
    CSpercent10 = ("CS%10", "last"),
    Whiffpercent5 = ("Whiff%5", "last"),
    Whiffpercent10 = ("Whiff%10", "last"),
    CSWpercent5 = ("CSW%5", "last"),
    CSWpercent10 = ("CSW%10", "last")).reset_index().fillna(0)

Season4.rename(columns={col: col.replace('percent', '%') for col in Season4.columns if 'percent' in col}, inplace=True)
Season4.rename(columns={col: col.replace('minus', '-') for col in Season4.columns if 'minus' in col}, inplace=True)

In [ ]:
BatterSeason1 = savant2024.groupby(["game_date", "BatterTeam", "away_team", "home_team", "inning", "at_bat_number", "MLBNAME", "player_name", "p_throws"]).agg(
    Pitches = ("pitch_number", "size"),
    PA = ('events', lambda x: (x.isin(['other_out', 'single', 'double', 'triple', 'home_run', 'strikeout', 'field_out', 'field_error', 'fielders_choice', 'double_play', 'fielders_choice_out', 'strikeout_double_play', 'triple_play', 'grounded_into_double_play'])).sum()),
    Outs = ('events', count_outs),
    FB = ('bb_type', lambda x: (x == 'fly_ball').sum()),
    HR = ('events', lambda x: (x == 'home_run').sum()),
    HBP = ('events', lambda x: (x == 'hit_by_pitch').sum()),
    Balls = ('description', lambda x: (x.isin(["ball", "hit_by_pitch", "blocked_ball"])).sum()),
    BB = ('events', lambda x: (x == 'walk').sum()),
    CS = ('description', lambda x: (x == 'called_strike').sum()),
    Whiff = ('description', lambda x: (x.isin(["swinging_strike", "swinging_strike_blocked", "foul_tip"])).sum()),
    Strikes = ('description', lambda x: (x.isin(["called_strike", "swinging_strike", "foul", "swinging_strike_blocked", "foul_tip"])).sum()),
    K = ('events', lambda x: (x == 'strikeout').sum()),
    xBA = ("estimated_ba_using_speedangle", "mean"),
    xwOBA = ("estimated_woba_using_speedangle", "mean"),
    wOBA = ("woba_value", "mean"),
    RunExp = ("delta_run_exp", "mean"),
    AwayRunsScored = ("AwayRunsScored", "sum"),
    HomeRunsScored = ("HomeRunsScored", "sum")).reset_index().fillna(0)

In [ ]:
BatterSeason2 = BatterSeason1.groupby(["game_date", "BatterTeam", "away_team", "home_team", "p_throws"]).agg(
    Pitches=("Pitches", "sum"),
    AB=("at_bat_number", "size"),
    PA=("PA", "sum"),
    Outs=("Outs", "sum"),
    FB=("FB", "sum"),
    HR=("HR", "sum"),
    HBP=("HBP", "sum"),
    Balls=("Balls", "sum"),
    BB=("BB", "sum"),
    CS=("CS", "sum"),
    Whiff=("Whiff", "sum"),
    Strikes=("Strikes", "sum"),
    K=("K", "sum"),
    xBA=("xBA", "mean"),
    xwOBA=("xwOBA", "mean"),
    wOBA=("wOBA", "mean"),
    RunExp=("RunExp", "mean"),
    AwayRunsScored=("AwayRunsScored", "sum"),
    HomeRunsScored=("HomeRunsScored", "sum")).reset_index().fillna(0)

BatterSeason2['RA'] = np.where((BatterSeason2['HomeRunsScored'] > 0) & (BatterSeason2['BatterTeam'] == BatterSeason2['home_team']),
    BatterSeason2["HomeRunsScored"], np.where((BatterSeason2['AwayRunsScored'] > 0) & (BatterSeason2['BatterTeam'] == BatterSeason2['away_team']), BatterSeason2["AwayRunsScored"], 0))
BatterSeason2["R/9"] = (27 * BatterSeason2["RA"] / BatterSeason2["Outs"])
BatterSeason2 = BatterSeason2.drop(columns=["AwayRunsScored", "HomeRunsScored", "RA"])

In [ ]:
BatterSeason2['FIP'] = (13 * BatterSeason2['HR'] + 3 * (BatterSeason2['BB'] + BatterSeason2['HBP']) - 2 * BatterSeason2['K']) / (BatterSeason2['Outs'] / 3) + 3.137
BatterSeason2['xFIP'] = (13 * (BatterSeason2['FB'] * (lgHR/lgFB * 0.58)) + 3 * (BatterSeason2['BB'] + BatterSeason2['HBP']) - 2 * BatterSeason2['K']) / (BatterSeason2['Outs'] / 3) + 3.137

BatterSeason2['K%'] = round((BatterSeason2['K'] / BatterSeason2['AB']) * 100, 2)
BatterSeason2['BB%'] = round((BatterSeason2['BB'] / BatterSeason2['AB']) * 100, 2)
BatterSeason2['K-BB%'] = BatterSeason2["K%"] - BatterSeason2["BB%"]
BatterSeason2['Ball%'] = round((BatterSeason2['Balls'] / BatterSeason2['Pitches']) * 100, 2)
BatterSeason2['Strike%'] = round((BatterSeason2['Strikes'] / BatterSeason2['Pitches']) * 100, 2)
BatterSeason2['CS%'] = round((BatterSeason2['CS'] / BatterSeason2['Pitches']) * 100, 2)
BatterSeason2['Whiff%'] = round((BatterSeason2['Whiff'] / BatterSeason2['Pitches']) * 100, 2)
BatterSeason2["CSW"] = BatterSeason2["CS"] + BatterSeason2["Whiff"]
BatterSeason2['CSW%'] = round((BatterSeason2['CSW'] / BatterSeason2['Pitches']) * 100, 2)
BatterSeason2 = BatterSeason2.drop(columns = ["CSW"])

In [ ]:
# Rolling 5 and 10 game expected batting averages
Season4['bxBA5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['xBA'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bxBA10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['xBA'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game expected wOBA averages
Season4['bxwOBA5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['xwOBA'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bxwOBA10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['xwOBA'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game wOBA averages
Season4['bwOBA5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['wOBA'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bwOBA10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['wOBA'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game RA averages
Season4['bRS5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['R/9'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bRS10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['R/9'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game FIP averages
Season4['bFIP5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['FIP'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bFIP10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['FIP'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game xFIP averages
Season4['bxFIP5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['xFIP'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bxFIP10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['xFIP'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game K% averages
Season4['bK%5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['K%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bK%10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['K%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game BB% averages
Season4['bBB%5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['BB%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bBB%10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['BB%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game K-BB% averages
Season4['bK-BB%5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['K-BB%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bK-BB%10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['K-BB%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 game Ball% averages
Season4['bBall%5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['Ball%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 game Strike% averages
Season4['bStrike%5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['Strike%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game Called Strike% averages
Season4['bCS%5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['CS%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bCS%10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['CS%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game Whiff% averages
Season4['bWhiff%5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['Whiff%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bWhiff%10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['Whiff%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)
# Rolling 5 and 10 game Called Strike plus Whiff% averages
Season4['bCSW%5'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['CSW%'].rolling(window=window_size5, min_periods=1).mean().reset_index(level=[0,1], drop=True)
Season4['bCSW%10'] = BatterSeason2.groupby(['BatterTeam', "p_throws"])['CSW%'].rolling(window=window_size10, min_periods=1).mean().reset_index(level=[0,1], drop=True)

## Joining both the pitcher averages and the batter rolling averages

In [ ]:
TodaysData.dropna(subset=['SP'], inplace=True)
TodaysData1 = pd.merge(TodaysData, Season4[['SP', 'p_throws', 'Starts', 'Pitches', 'AB', 'PA', 'Outs', "Hits", 'HR', 'BB',
       'K', 'RA', 'xBA', 'xwOBA', 'wOBA', 'RunExp', 'FIP', 'xFIP', 'K%', 'BB%',
       'K-BB%', 'Pitches5', 'Outs5', 'Outs10', 'xBA5', 'xBA10', 'xwOBA5',
       'xwOBA10', 'wOBA5', 'wOBA10', 'RA5', 'RA10', 'FIP5', 'FIP10', 'xFIP5',
       'xFIP10', 'K%5', 'K%10', 'BB%5', 'BB%10', 'K-BB%5', 'K-BB%10', 'Ball%5',
       'Strike%5', 'CS%5', 'CS%10', 'Whiff%5', 'Whiff%10', 'CSW%5', 'CSW%10',
       'bxBA5', 'bxBA10', 'bxwOBA5', 'bxwOBA10', 'bwOBA5', 'bwOBA10', 'bRS5',
       'bRS10', 'bFIP5', 'bFIP10', 'bxFIP5', 'bxFIP10', 'bK%5', 'bK%10',
       'bBB%5', 'bBB%10', 'bK-BB%5', 'bK-BB%10', 'bBall%5', 'bStrike%5',
       'bCS%5', 'bCS%10', 'bWhiff%5', 'bWhiff%10', 'bCSW%5', 'bCSW%10']], left_on = ['SP'], right_on = ['SP'], how = 'left')

# If no 2024 savant data exists then gives them the league averages from 2022-23
TrainMeans = Train3.drop(['BatterTeam', 'RoadTeam', "HomeTeam", "SP", "p_throws"], axis=1).mean()
TodaysData1 = TodaysData1.fillna(TrainMeans)
TodaysData1 = TodaysData1.replace([float('inf'), -float('inf')], 5)

# Min max scaling to normalize the variables

In [ ]:
Train4 = Train3.copy()

# Identify numeric columns
numeric_columnsTrain = Train4.select_dtypes(include=['float64', 'int64']).columns
numeric_columnsToday = TodaysData1.select_dtypes(include=['float64', 'int64']).columns

# Initialize MinMaxScaler
scaler = MinMaxScaler()
# Fit and transform numeric columns
Train4[numeric_columnsTrain] = scaler.fit_transform(Train4[numeric_columnsTrain])
TodaysData1[numeric_columnsTrain] = scaler.fit_transform(TodaysData1[numeric_columnsToday])

## Encodes the teams and players allowing to be fed into the algorithms

In [ ]:
# Ensure Train5 and TodaysData2 are copies of Train4 and TodaysData1 respectively
Train5 = Train4.copy()
TodaysData2 = TodaysData1.copy()

def strip_2_from_columns(df, columns):
    for col in columns:
        df[col] = df[col].astype(str).str.replace('2', '', regex=False)
    return df

# Apply the function to TodaysData2
TodaysData2 = strip_2_from_columns(TodaysData2, ['BatterTeam', 'HomeTeam', 'RoadTeam'])

# Dictionary to store the label encoders
label_encoders = {}

# Encode non-numeric columns in Train4
non_numeric_columns_train = Train5.select_dtypes(exclude=['float64', 'int64']).columns
for col in non_numeric_columns_train:
    label_encoder = LabelEncoder()
    Train5[col] = label_encoder.fit_transform(Train5[col])
    label_encoders[col] = label_encoder

# Ensure all non-numeric columns in Train4 are in TodaysData1
for col in non_numeric_columns_train:
    if col not in TodaysData2.columns:
        print(f"Warning: Column {col} from training data is not present in today's data.")
        # Adding the missing column with a default value
        TodaysData2[col] = 536

# Encode non-numeric columns in TodaysData1 using the same encoders
non_numeric_columns_today = TodaysData2.select_dtypes(exclude=['float64', 'int64']).columns
for col in non_numeric_columns_today:
    if col in label_encoders:
        label_encoder = label_encoders[col]
        unique_values = set(label_encoder.classes_)
        encoded_values = []
        for item in TodaysData2[col]:
            if item in unique_values:
                encoded_values.append(label_encoder.transform([item])[0])
            else:
                encoded_values.append(536)  # Using 536 as a placeholder for unknown categories
        TodaysData2[col] = encoded_values
    else:
        print(f"Warning: Column {col} is not present in the training data.")
        # Fit a new label encoder for columns not present in Train4, but be cautious with this
        label_encoder = LabelEncoder()
        TodaysData2[col] = label_encoder.fit_transform(TodaysData2[col])
        label_encoders[col] = label_encoder

- Used Lasso Regression to find optimal features for each model used
- Also looked at the VIF to get rid of features that had high multicollinearity
- Repeated this process until satisfied

## K model

In [ ]:
kData = Train5[["RoadTeam", 'HomeTeam', 'SP', 'p_throws', 'Pitches', "PA", 'Outs', 'Hits', 'HR', 'xBA', 'xwOBA', 'wOBA', 'RA', 'FIP', 'xFIP', 'K%', 'BB%', 'Pitches5', 'xBA10', 'RA5', 'RA10', 'FIP5', 'FIP10', 'xFIP5', 'K%5', 'Strike%5', 'Whiff%10', 'CSW%5', 'bxBA5', 'bxwOBA10', 'bxFIP5', 'bBB%5', 'bK-BB%10', 'bBall%5', 'bCS%10', "K"]]

kTodaysData = TodaysData2[["RoadTeam", 'HomeTeam', 'SP', 'p_throws', 'Pitches', "PA", 'Outs', 'Hits', 'HR', 'xBA', 'xwOBA', 'wOBA', 'RA', 'FIP', 'xFIP', 'K%', 'BB%', 'Pitches5', 'xBA10', 'RA5', 'RA10', 'FIP5', 'FIP10', 'xFIP5', 'K%5', 'Strike%5', 'Whiff%10', 'CSW%5', 'bxBA5', 'bxwOBA10', 'bxFIP5', 'bBB%5', 'bK-BB%10', 'bBall%5', 'bCS%10', "K", "Starts"]]

kData.loc[:,"K"] = Train3["K"]
kTodaysData.loc[:,"K"] = TodaysData1["K"]

TrainFeatures = kData.drop(columns = ["K"]).values.reshape(-1, 31)
TrainLabel = kData["K"].values.ravel()
TodayFeatures = kTodaysData.drop(columns = ["Starts", "K"]).values.reshape(-1, 31)

# Define the model
input_layer = Input(shape=(31,))  # Adjusted to match the number of features
x = Dense(units=160, activation='relu', kernel_regularizer=l2(0.0001119799781856557))(input_layer)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)
x = Dense(units=32, activation='relu', kernel_regularizer=l2(0.0025661817326560264))(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
output_layer = Dense(units=1)(x)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0021599950963650445), loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(TrainFeatures, TrainLabel, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

NNpred = model.predict(TodayFeatures).flatten()
TodaysData2["xK"] = NNpred

## BB Model

In [ ]:
bbData = Train5[['RoadTeam', 'HomeTeam', 'p_throws', 'Pitches', 'PA', 'Outs', 'Hits', 'K', 'xBA', 'xwOBA', 'RA', 'xFIP', 'BB%', 'Pitches5', 'Outs5', 'xBA10', 'xwOBA5', 'K%5', 'Ball%5', 'Strike%5', 'bxBA10', 'bxFIP10', 'bWhiff%10', 'bCSW%5', "BB"]]

bbTodaysData = TodaysData2[['RoadTeam', 'HomeTeam', 'p_throws', 'Pitches', 'PA', 'Outs', 'Hits', 'K', 'xBA', 'xwOBA', 'RA', 'xFIP', 'BB%', 'Pitches5', 'Outs5', 'xBA10', 'xwOBA5', 'K%5', 'Ball%5', 'Strike%5', 'bxBA10', 'bxFIP10', 'bWhiff%10', 'bCSW%5', "BB", "Starts"]]

bbData.loc[:,"BB"] = Train3["BB"]
bbTodaysData.loc[:,"BB"] = TodaysData1["BB"]

TrainFeatures = kData.drop(columns = ["K"]).values.reshape(-1, 24)
TrainLabel = kData["K"].values.ravel()
TodayFeatures = kTodaysData.drop(columns = ["Starts", "K"]).values.reshape(-1, 24)

# Define the model
input_layer = Input(shape=(24,))  # Adjusted to match the number of features
x = Dense(units=160, activation='relu', kernel_regularizer=l2(0.0001119799781856557))(input_layer)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)
x = Dense(units=32, activation='relu', kernel_regularizer=l2(0.0025661817326560264))(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
output_layer = Dense(units=1)(x)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0021599950963650445), loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(TrainFeatures, TrainLabel, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

NNpred = model.predict(TodayFeatures).flatten()
TodaysData2["xBB"] = NNpred

## Hits Model

In [ ]:
hitsData = Train5[['SP', 'p_throws', 'Pitches', 'PA', 'Outs', 'HR', 'BB', 'K', 'RA', 'xBA', 'xwOBA', 'wOBA', 'FIP', 'xFIP', 'Pitches5', 'RA5', 'Ball%5', 'Strike%5', "Hits"]]

hitsTodaysData = TodaysData2[['SP', 'p_throws', 'Pitches', 'PA', 'Outs', 'HR', 'BB', 'K', 'RA', 'xBA', 'xwOBA', 'wOBA', 'FIP', 'xFIP', 'Pitches5', 'RA5', 'Ball%5', 'Strike%5', "Hits", "Starts"]]

hitsData.loc[:,"Hits"] = Train3["Hits"]
hitsTodaysData.loc[:,"Hits"] = TodaysData1["Hits"]

TrainFeatures = hitsData.drop(columns = ["Hits"]).values.reshape(-1, 18)
TrainLabel = hitsData["Hits"].values.ravel()
TodayFeatures = hitsTodaysData.drop(columns = ["Starts", "Hits"]).values.reshape(-1, 18)

# Define the model
input_layer = Input(shape=(18,))  # Adjusted to match the number of features
x = Dense(units=160, activation='relu', kernel_regularizer=l2(0.0001119799781856557))(input_layer)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)
x = Dense(units=32, activation='relu', kernel_regularizer=l2(0.0025661817326560264))(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
output_layer = Dense(units=1)(x)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0021599950963650445), loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(TrainFeatures, TrainLabel, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

NNpred = model.predict(TodayFeatures).flatten()
TodaysData2["xHits"] = NNpred

## RA Model

- Model has an error of about 1, just hard to predict from

In [ ]:
raData = Train5[['SP', 'p_throws', 'AB', 'Outs', 'Hits', 'HR', 'BB', 'K', 'xBA', 'RunExp', 'Pitches5', 'Outs5', 'xBA5', 'wOBA5', 'RA5', "RA"]]

raTodaysData = TodaysData2[['SP', 'p_throws', 'AB', 'Outs', 'Hits', 'HR', 'BB', 'K', 'xBA', 'RunExp', 'Pitches5', 'Outs5', 'xBA5', 'wOBA5', 'RA5', "RA", "Starts"]]

raData.loc[:,"RA"] = Train3["RA"]
raTodaysData.loc[:,"RA"] = TodaysData1["RA"]

TrainFeatures = raData.drop(columns = ["RA"]).values.reshape(-1, 15)
TrainLabel = raData["RA"].values.ravel()
TodayFeatures = raTodaysData.drop(columns = ["Starts", "RA"]).values.reshape(-1, 15)

# Define the model
input_layer = Input(shape=(15,))  # Adjusted to match the number of features
x = Dense(units=160, activation='relu', kernel_regularizer=l2(0.0001119799781856557))(input_layer)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)
x = Dense(units=32, activation='relu', kernel_regularizer=l2(0.0025661817326560264))(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
output_layer = Dense(units=1)(x)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0021599950963650445), loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(TrainFeatures, TrainLabel, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

NNpred = model.predict(TodayFeatures).flatten()
TodaysData2["xRA"] = NNpred

## IP model

In [ ]:
outsData = Train5[['SP', 'AB', 'PA', 'Hits', 'BB', 'RA', 'wOBA', 'Outs5', "Outs"]]

outsTodaysData = TodaysData2[['SP', 'AB', 'PA', 'Hits', 'BB', 'RA', 'wOBA', 'Outs5', "Outs", "Starts"]]

outsData.loc[:,"Outs"] = Train3["Outs"]
outsTodaysData.loc[:,"Outs"] = TodaysData1["Outs"]

TrainFeatures = outsData.drop(columns = ["Outs"]).values.reshape(-1, 8)
TrainLabel = outsData["Outs"].values.ravel()
TodayFeatures = outsTodaysData.drop(columns = ["Starts", "Outs"]).values.reshape(-1, 8)

# Define the model
input_layer = Input(shape=(8,))  # Adjusted to match the number of features
x = Dense(units=160, activation='relu', kernel_regularizer=l2(0.0001119799781856557))(input_layer)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)
x = Dense(units=32, activation='relu', kernel_regularizer=l2(0.0025661817326560264))(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
output_layer = Dense(units=1)(x)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0021599950963650445), loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(TrainFeatures, TrainLabel, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

NNpred = model.predict(TodayFeatures).flatten()
TodaysData2["xOuts"] = NNpred

## Reverse encodes today's data so it can be understood

In [ ]:
# Ensure Train5 and TodaysData2 are copies of Train4 and TodaysData1 respectively
Train5 = Train3.copy()
TodaysData2 = TodaysData1.copy()

def strip_2_from_columns(df, columns):
    for col in columns:
        df[col] = df[col].astype(str).str.replace('2', '', regex=False)
    return df

# Apply the function to TodaysData2
TodaysData2 = strip_2_from_columns(TodaysData2, ['BatterTeam', 'HomeTeam', 'RoadTeam'])

# Dictionary to store the label encoders
label_encoders = {}

# Encode non-numeric columns in Train4
non_numeric_columns_train = Train5.select_dtypes(exclude=['float64', 'int64']).columns
for col in non_numeric_columns_train:
    label_encoder = LabelEncoder()
    Train5[col] = label_encoder.fit_transform(Train5[col])
    label_encoders[col] = label_encoder

# Ensure all non-numeric columns in Train4 are in TodaysData1
for col in non_numeric_columns_train:
    if col not in TodaysData2.columns:
        print(f"Warning: Column {col} from training data is not present in today's data.")
        # Adding the missing column with a default value
        # There are 535 unique starting pitchers in the training set
        TodaysData2[col] = 536

# Encode non-numeric columns in TodaysData1 using the same encoders
non_numeric_columns_today = TodaysData2.select_dtypes(exclude=['float64', 'int64']).columns
for col in non_numeric_columns_today:
    if col in label_encoders:
        label_encoder = label_encoders[col]
        unique_values = set(label_encoder.classes_)
        encoded_values = []
        for item in TodaysData2[col]:
            if item in unique_values:
                encoded_values.append(label_encoder.transform([item])[0])
            else:
                encoded_values.append(536)  # Using 536 as a placeholder for unknown categories
        TodaysData2[col] = encoded_values
    else:
        print(f"Warning: Column {col} is not present in the training data.")
        # Fit a new label encoder for columns not present in Train4, but be cautious with this
        label_encoder = LabelEncoder()
        TodaysData2[col] = label_encoder.fit_transform(TodaysData2[col])
        label_encoders[col] = label_encoder

## Attempting to mess with Neural Networks and see if their results are better

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from keras.regularizers import l2

# Assuming TrainFeatures and TrainLabel are your prepared datasets
# Standardize the features
scaler = StandardScaler()
TrainFeatures = scaler.fit_transform(Train5.drop(columns=["K"]).values.reshape(-1, 77))
TrainLabel = Train5["K"].values
TodayFeatures = scaler.transform(TodaysData2.drop(columns=["Starts", "K"]).values.reshape(-1, 77))

# Define the model with adjustments
input_layer = Input(shape=(77,))
x = Dense(units=128, activation='relu', kernel_regularizer=l2(0.01))(input_layer)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(units=64, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(units=32, activation='relu', kernel_regularizer=l2(0.01))(x)
output_layer = Dense(units=1, activation = "softplus")(x)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.1), loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
model.fit(TrainFeatures, TrainLabel, epochs=20, batch_size=64, validation_split=0.1, callbacks=[early_stopping], verbose=1)

RFpred = model.predict(TodayFeatures)
TodaysData2["xK"] = RFpred

In [ ]:
for col in non_numeric_columns_today:
    if col in label_encoders:
        label_encoder = label_encoders[col]
        # Handling default value of 536
        TodaysData2[col] = TodaysData2[col].apply(lambda x: label_encoder.inverse_transform([x])[0] if x != 536 else np.nan)

TodaysData2['SP'] = TodaysData2['SP'].fillna(TodaysData1['SP'])

In [ ]:
def add_2_to_duplicates(df):
    mask = df.duplicated(subset=['BatterTeam', 'RoadTeam', 'HomeTeam'], keep='first')
    
    df.loc[mask, 'BatterTeam'] += '2'
    df.loc[mask, 'RoadTeam'] += '2'
    df.loc[mask, 'HomeTeam'] += '2'
    
    return df

TodaysData2 = add_2_to_duplicates(TodaysData2)

In [ ]:
TodaysData2[["BatterTeam", "SP", "Starts", "K", "xK"]].sort_values("xK", ascending = True)

## Creates simple dataset to see the expected stats of all the predicted metrics

In [ ]:
# Performs a two "sample" z test to see the likelihood of a quality start occurring
def row_z_score(row):
    x_bar1 = row["xRA"]
    x_bar2 = row["xOuts"]
    mu1 = 3
    mu2 = 18
    sigma1 = Train2["RA"].std()
    sigma2 = Train2["Outs"].std()
    n1 = len(TodaysData2)
    n2 = n1
    
    # Calculate z-scores
    z_score_xRA = (x_bar1 - mu1) / (sigma1 / np.sqrt(n1))
    z_score_xOuts = (x_bar2 - mu2) / (sigma2 / np.sqrt(n2))
    
    # Calculate probabilities, want less than
    prob_xRA_less_than_mu1 = 1 - stats.norm.cdf(z_score_xRA)
    prob_xOuts_greater_than_mu2 = stats.norm.cdf(z_score_xOuts)

    # Combine probabilities by multiplying it
    combined_prob = prob_xRA_less_than_mu1 * prob_xOuts_greater_than_mu2
    return combined_prob

# Calculate the chance of a quality start occurring and finds the xQS points
percent = TodaysData2.apply(row_z_score, axis=1)
xQS = percent * 5

# Adds the xQS to the xFS
TodaysData2.loc[:, "xFS"] = (xQS + TodaysData2["xK"] * 3 + TodaysData2["xOuts"] - TodaysData2["xRA"] * 3)
TodaysData3 = TodaysData2[["BatterTeam", "SP", "Starts", "xK", "xBB", "xHA", "xRA", "xOuts", "xFS"]].round(2)
TodaysData3.sort_values("xFS")